In [2]:

import os
os.getcwd() 
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
from test import *
warnings.filterwarnings("ignore")
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch_optimizer as optim
from transformers.optimization import Adafactor, AdafactorSchedule
import torch.backends.cudnn as cudnn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler
from torch.autograd import Variable
import logging
import sys
import transformers
from basic_model import *
import time
import argparse
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import string
from cycle import *
from train import *
from utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-small')
def d(l):
    return tokenizer.batch_decode(l,skip_special_tokens=True)
def en(l):
    return tokenize(l,tokenizer,512,True)

In [15]:
fake_B = torch.zeros((3,4,5))
fake_B[0][0][1] = 1
fake_B[0][1][0] = 1
fake_B[0][2][3] = 1
fake_B

tensor([[[0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])

In [16]:

a  = (fake_B[:, :,0] != 1).long()

In [3]:
model = torch.load('./model/D_B.pt').eval()

In [12]:
#real sentences
x = [[10180,   351, 19810,     7,    24,     6,    38,     8,  8160,  8675,
            13,     8,  1611, 15375,     6,    62,   225,     6,    11,  5071,
           398,     6,  2173,   981, 12361,   485,    11, 13567,    16,     8,
          1028,  5808,     7,  1194,    11,  6572,    53,    13,    48,   540,
             6, 10321,    69, 12123,     7,    11,   811,     7,  9098,    12,
             3,  9582,   125,    33,   801,    38,    96, 12097,    21,   540,
           121, 15600,    16,     8,  5334,    18, 14259,   433,     5,     1,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]]

             #delte 0 give huge diff
       
x = torch.Tensor(x).long().cuda()
print(x.shape)
x_attn = (x>0.5).long()

print('\t\t---------embedding')
x_emb = model.embedding(x)
print(x_emb.shape)
print(x_emb)# same for padding

print('\t\t---------encode')
distr = model.encoder(inputs_embeds=x_emb,attention_mask=x_attn).last_hidden_state#(bs,sentence length,512)
print(distr.shape)
print(distr)

print('\t\t---------mask the vector')
x_attn_unsq= x_attn.unsqueeze(-1)
distr = torch.mul(distr,x_attn_unsq)#previously
print(distr.shape)
print(distr)


print('\t\t---------mean of the unmask ')
distr = distr[:,0,:]
print(distr.shape)
print(distr)

print('\t\t---------classifer ')
ret =  model.classifier(distr)#(bs,1)
print(ret)

torch.Size([1, 144])
		---------embedding
torch.Size([1, 144, 512])
tensor([[[  6.0501,  -7.9552,  14.4621,  ..., -21.1924,   4.0715,  12.5375],
         [ 23.8152,  -2.8088, -15.9855,  ...,  25.2238,  -6.3611,   2.1363],
         [  5.3305, -38.3739,  46.3184,  ...,  -6.1115, -31.1421,  -2.6996],
         ...,
         [ -1.9571,   0.1690,  -7.0790,  ...,  -0.4705,   2.6937,  -2.8559],
         [ -1.9571,   0.1690,  -7.0790,  ...,  -0.4705,   2.6937,  -2.8559],
         [ -1.9571,   0.1690,  -7.0790,  ...,  -0.4705,   2.6937,  -2.8559]]],
       device='cuda:0', grad_fn=<EmbeddingBackward0>)
		---------encode
torch.Size([1, 144, 512])
tensor([[[ 0.4458, -0.1212,  0.0547,  ..., -0.3194,  0.0034, -0.1237],
         [ 0.5885, -0.2483,  0.2204,  ..., -0.1367,  0.0660, -0.0868],
         [ 0.0803,  0.0777,  0.3199,  ...,  0.1108, -0.1654,  0.0257],
         ...,
         [ 0.2039,  0.0961,  0.2980,  ..., -0.0319, -0.1689, -0.1566],
         [ 0.2093,  0.0700,  0.3007,  ..., -0.0212, -0.157

In [11]:
#real sentences
x = [[10180,   351, 19810,     7,    24,     6,    38,     8,  8160,  8675,
            13,     8,  1611, 15375,     6,    62,   225,     6,    11,  5071,
           398,     6,  2173,   981, 12361,   485,    11, 13567,    16,     8,
          1028,  5808,     7,  1194,    11,  6572,    53,    13,    48,   540,
             6, 10321,    69, 12123,     7,    11,   811,     7,  9098,    12,
             3,  9582,   125,    33,   801,    38,    96, 12097,    21,   540,
           121, 15600,    16,     8,  5334,    18, 14259,   433,     5,     1,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]

             #delte 0 give huge diff
       
x = torch.Tensor(x).long().cuda()
print(x.shape)
x_attn = (x>0.5).long()

print('\t\t---------embedding')
x_emb = model.embedding(x)
print(x_emb.shape)
print(x_emb)# same for padding

print('\t\t---------encode')
distr = model.encoder(inputs_embeds=x_emb,attention_mask=x_attn).last_hidden_state#(bs,sentence length,512)
print(distr.shape)
print(distr)

print('\t\t---------mask the vector')
x_attn_unsq= x_attn.unsqueeze(-1)
distr = torch.mul(distr,x_attn_unsq)#previously
print(distr.shape)
print(distr)


print('\t\t---------mean of the unmask ')
distr = distr[:,0,:]
print(distr.shape)
print(distr)

print('\t\t---------classifer ')
ret =  model.classifier(distr)#(bs,1)
print(ret)

torch.Size([1, 90])
		---------embedding
torch.Size([1, 90, 512])
tensor([[[  6.0501,  -7.9552,  14.4621,  ..., -21.1924,   4.0715,  12.5375],
         [ 23.8152,  -2.8088, -15.9855,  ...,  25.2238,  -6.3611,   2.1363],
         [  5.3305, -38.3739,  46.3184,  ...,  -6.1115, -31.1421,  -2.6996],
         ...,
         [ -1.9571,   0.1690,  -7.0790,  ...,  -0.4705,   2.6937,  -2.8559],
         [ -1.9571,   0.1690,  -7.0790,  ...,  -0.4705,   2.6937,  -2.8559],
         [ -1.9571,   0.1690,  -7.0790,  ...,  -0.4705,   2.6937,  -2.8559]]],
       device='cuda:0', grad_fn=<EmbeddingBackward0>)
		---------encode
torch.Size([1, 90, 512])
tensor([[[ 0.4459, -0.1211,  0.0547,  ..., -0.3193,  0.0034, -0.1236],
         [ 0.5885, -0.2484,  0.2204,  ..., -0.1367,  0.0661, -0.0868],
         [ 0.0802,  0.0776,  0.3199,  ...,  0.1108, -0.1654,  0.0257],
         ...,
         [ 0.2089,  0.0910,  0.2934,  ..., -0.0097, -0.1524, -0.1347],
         [ 0.2074,  0.0918,  0.2950,  ..., -0.0076, -0.1534, 

In [45]:

# x_emb = model.embedding(x_)#
# print('x_emb.shape',x_emb.shape)
# distr = model.encoder(inputs_embeds=x_emb).last_hidden_state#(bs,sentence length,512)
# print('distr.shape',distr.shape)
# x_attn= x_attn.unsqueeze(-1)
# print('x_attn.shape',x_attn.shape)
# distr = torch.mul(distr,x_attn)#previously

# print('mutil',torch.sum(distr,1))
# print('count',torch.sum(x_attn,1))
# distr = torch.sum(distr,1)/torch.sum(x_attn,1)#(bs,512)
# print(torch.mean(distr,-1))
# print('mea',distr.shape)

# ret =  model.classifier(distr)#(bs,1)
# print(ret)
# ret = model.relu(ret)#(bs,1)
# print(ret)



a_pred_dis = model(x_,x_attn)
a_pred_dis

tensor([[0.9008]], device='cuda:0', grad_fn=<ReluBackward0>)

,attention_mask=x_attn is essential for the encoder inpuit

## We previously use torch.mean to (x after embedding)*x_attn but the mean will be small for the long sentences, now we use torch.sum(distr,1)/torch.sum(x_attn,1)

## We can see that the D judge the sentence by it's length
## small output for the first part, but large for the latter part, so the longer the higher

## so we change the output to [batch,0,512] ie, onlythe first word's sumQ*V and than droppout and then classifier